In [18]:
import os
import glob
import numpy as np
import tensorflow as tf
from skimage.io import imread
from skimage import img_as_float
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, Add, Lambda, LayerNormalization, MultiHeadAttention, Dense, Layer, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt

# Load and preprocess images from a given folder
def load_images_from_folder(folder, size=(128, 128)):
    images = []
    for filename in glob.glob(os.path.join(folder, '*.png')):
        img = imread(filename)
        img = img_as_float(img)
        img_resized = tf.image.resize(img, size).numpy()
        images.append(img_resized)
    return np.array(images)

# Data preprocessing and augmentation pipeline
def preprocess_and_augment_data(images, augment=False):
    if augment:
        datagen = ImageDataGenerator(
            rotation_range=15,
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0.01,
            zoom_range=[0.9, 1.25],
            horizontal_flip=True,
            fill_mode='reflect'
        )
        augmented_images = datagen.flow(images, batch_size=len(images), shuffle=False)
        augmented_images = next(augmented_images)  # Fetch the augmented images
        return augmented_images
    return images

# Function to load datasets from base folder (for both training and testing)
def load_and_preprocess_datasets(base_folder, size=(128, 128), augment=False):
    rainy_images_train, clear_images_train = [], []
    rainy_images_test, clear_images_test = [], []

    datasets = ['Rain200L', 'Rain200H']  # Add more datasets if necessary

    for dataset in datasets:
        train_folder = os.path.join(base_folder, dataset, 'train')
        test_folder = os.path.join(base_folder, dataset, 'test')

        # Load training data
        rainy_train_folder = os.path.join(train_folder, 'input')
        clear_train_folder = os.path.join(train_folder, 'target')
        rainy_images_train.extend(load_images_from_folder(rainy_train_folder, size))
        clear_images_train.extend(load_images_from_folder(clear_train_folder, size))

        # Load testing data
        rainy_test_folder = os.path.join(test_folder, 'input')
        clear_test_folder = os.path.join(test_folder, 'target')
        rainy_images_test.extend(load_images_from_folder(rainy_test_folder, size))
        clear_images_test.extend(load_images_from_folder(clear_test_folder, size))

    # Convert lists to numpy arrays
    rainy_images_train = np.array(rainy_images_train)
    clear_images_train = np.array(clear_images_train)
    rainy_images_test = np.array(rainy_images_test)
    clear_images_test = np.array(clear_images_test)

    # Apply data augmentation to training data (if augment is True)
    if augment:
        rainy_images_train = preprocess_and_augment_data(rainy_images_train, augment=True)
        clear_images_train = preprocess_and_augment_data(clear_images_train, augment=True)

    return (rainy_images_train, clear_images_train), (rainy_images_test, clear_images_test)

# Advanced Dynamic Pyramid Model
def advanced_dynamic_pyramid_model(input_shape, num_scales=3):
    inputs = Input(shape=input_shape)
    pyramid_features = []

    for scale in range(num_scales):
        scale_factor = 2 ** scale
        x_scaled = Lambda(lambda x: tf.image.resize(x, 
                            [input_shape[0] // scale_factor, input_shape[1] // scale_factor]))(inputs)
        
        kernel_size = 3 + scale
        x_conv = Conv2D(64 * scale_factor, (kernel_size, kernel_size), activation='relu', padding='same')(x_scaled)
        x_scaled_conv = Conv2D(64 * scale_factor, (1, 1), padding='same')(x_scaled)
        x_conv = Add()([x_conv, x_scaled_conv])
        
        x_resized = Lambda(lambda x: tf.image.resize(x, 
                              [input_shape[0], input_shape[1]]))(x_conv)
        
        pyramid_features.append(x_resized)

    fused_features = Lambda(lambda x: tf.concat(x, axis=-1))(pyramid_features)
    return inputs, fused_features

# Custom Layer: Window Partition Layer
class WindowPartitionLayer(Layer):
    def __init__(self, window_size):
        super(WindowPartitionLayer, self).__init__()
        self.window_size = window_size

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        windowed = tf.image.extract_patches(
            inputs,
            sizes=[1, self.window_size, self.window_size, 1],
            strides=[1, self.window_size, self.window_size, 1],
            rates=[1, 1, 1, 1],
            padding='VALID'
        )
        windowed = tf.reshape(windowed, (batch_size, -1, self.window_size * self.window_size, tf.shape(inputs)[-1]))
        return windowed

# Custom Layer: Patch Merge Layer
class PatchMergeLayer(Layer):
    def __init__(self, window_size):
        super(PatchMergeLayer, self).__init__()
        self.window_size = window_size

    def call(self, inputs):
        batch_size, num_patches, flattened_window_size, channels = tf.shape(inputs)[0], tf.shape(inputs)[1], tf.shape(inputs)[2], tf.shape(inputs)[3]
        merged = tf.reshape(inputs, (batch_size, int(num_patches**0.5), int(num_patches**0.5), self.window_size, self.window_size, channels))
        merged = tf.reshape(merged, (batch_size, int(num_patches**0.5) * self.window_size, int(num_patches**0.5) * self.window_size, channels))
        return merged

class ModifiedSwinTransformerBlock(Layer):
    def __init__(self, initial_window_size=4, num_heads=4, key_dim=64, num_recursions=3):
        super(ModifiedSwinTransformerBlock, self).__init__()
        self.initial_window_size = initial_window_size
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.num_recursions = num_recursions

    def call(self, fused_features):
        original_shape = tf.shape(fused_features)  # Shape: (batch_size, 128, 128, 448)
        x = self.recursive_block(fused_features, self.initial_window_size, 2, self.num_recursions, original_shape)
        return x

    def recursive_block(self, x, window_size, iteration, recursion, original_shape):
        if recursion == 0:
            return x
        
        for _ in range(iteration):
            partition_layer = WindowPartitionLayer(window_size)
            windows = partition_layer(x)  # Shape: (batch_size, num_patches, window_size * window_size, channels)

            attention = MultiHeadAttention(num_heads=self.num_heads, key_dim=self.key_dim)(windows, windows)  # Shape: (batch_size, num_patches, window_size * window_size, channels)

            # Reshape attention output back to original window shape
            attention_reshaped = tf.reshape(attention, tf.shape(windows))  # Shape: (batch_size, num_patches, window_size * window_size, channels)
            x_add = Add()([windows, attention_reshaped])  # Shape: (batch_size, num_patches, window_size * window_size, channels)

            # Normalization and Feed Forward Network
            x_norm_ffn = LayerNormalization(axis=-1)(x_add)  # Shape: (batch_size, num_patches, window_size * window_size, channels)
            x_ffn = Dense(128, activation='relu')(x_norm_ffn)  # Shape: (batch_size, num_patches, window_size * window_size, 128)
            x_ffn_out = Dense(tf.shape(x)[-1])(x_ffn)  # Shape: (batch_size, num_patches, window_size * window_size, channels)

            # Reverse window partitioning to original spatial dimensions
            x_reconstructed = self.window_reverse(x_ffn_out, original_shape)  # Ensure shape matches original

            # Update window size for the next iteration
            window_size *= 2  # Double the window size

        return self.recursive_block(x_reconstructed, window_size, iteration, recursion - 1, original_shape)

    def window_reverse(self, x, original_shape):
        # Implement reverse logic based on windowed output to reconstruct original shape
        # Ensure that the output shape matches original shape required for LayerNormalization
        batch_size = tf.shape(x)[0]
        return tf.reshape(x, (batch_size, original_shape[1], original_shape[2], -1))  # Ensure compatibility


# Image Restoration and Enhancement Block
def image_restoration_and_enhancement(x_out, num_classes=3):
    x = LayerNormalization()(x_out)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = UpSampling2D(size=(2, 2))(x)
    outputs = Conv2D(num_classes, (1, 1), activation='sigmoid')(x)
    return outputs

def build_model(input_shape):
    inputs, fused_features = advanced_dynamic_pyramid_model(input_shape)
    
    # Use the modified Swin Transformer block for processing
    transformer_block = ModifiedSwinTransformerBlock(initial_window_size=4, num_heads=4, key_dim=64, num_recursions=3)
    x_out = transformer_block(fused_features)

    # Image restoration and enhancement
    outputs = image_restoration_and_enhancement(x_out)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Load and preprocess the datasets
base_folder = '/kaggle/input/derainingdata/RainData'  # Update with your dataset path
(input_shape, num_classes) = (128, 128, 3), 3  # Example shape and number of classes
(rainy_train, clear_train), (rainy_test, clear_test) = load_and_preprocess_datasets(base_folder, size=(128, 128), augment=True)

# Build the model
model = build_model(input_shape)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Callbacks for training
callbacks = [
    ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss'),
    EarlyStopping(patience=5, restore_best_weights=True)
]

# Train the model
history = model.fit(rainy_train, clear_train, validation_split=0.1, epochs=50, batch_size=16, callbacks=callbacks)

# Evaluate the model on the test set
test_loss, test_mae = model.evaluate(rainy_test, clear_test)

# Visualize training history
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

# Function to visualize results
def visualize_results(rainy_images, clear_images, model):
    predictions = model.predict(rainy_images)

    for i in range(len(rainy_images)):
        plt.figure(figsize=(12, 6))

        plt.subplot(1, 3, 1)
        plt.title("Rainy Image")
        plt.imshow(rainy_images[i])

        plt.subplot(1, 3, 2)
        plt.title("Ground Truth")
        plt.imshow(clear_images[i])

        plt.subplot(1, 3, 3)
        plt.title("Model Prediction")
        plt.imshow(predictions[i])

        plt.show()

# Visualize some predictions
visualize_results(rainy_test[:5], clear_test[:5], model)



RuntimeError: Exception encountered when calling ModifiedSwinTransformerBlock.call().

[1mCould not automatically infer the output shape / dtype of 'modified_swin_transformer_block_10' (of type ModifiedSwinTransformerBlock). Either the `ModifiedSwinTransformerBlock.call()` method is incorrect, or you need to implement the `ModifiedSwinTransformerBlock.compute_output_spec() / compute_output_shape()` method. Error encountered:

Invalid dtype: <property object at 0x791c0d366a70>[0m

Arguments received by ModifiedSwinTransformerBlock.call():
  • args=('<KerasTensor shape=(None, 128, 128, 448), dtype=float32, sparse=False, name=keras_tensor_370>',)
  • kwargs=<class 'inspect._empty'>

In [ ]:
pip install torchsummary


In [ ]:
pip install torchinfo


# Swin transformer block

In [2]:
def swin_transformer_block(inputs):
    # Here you would implement your actual Swin Transformer logic
    
    
    # --------------------------------------------------------
    # Swin Transformer Code
    # --------------------------------------------------------

    import torch
    import torch.nn as nn
    import torch.utils.checkpoint as checkpoint
    from timm.models.layers import DropPath, to_2tuple, trunc_normal_

    try:
        import os, sys

        kernel_path = os.path.abspath(os.path.join('..'))
        sys.path.append(kernel_path)
        from kernels.window_process.window_process import WindowProcess, WindowProcessReverse

    except:
        WindowProcess = None
        WindowProcessReverse = None
        print("[Warning] Fused window process have not been installed. Please refer to get_started.md for installation.")

    class Mlp(nn.Module):
        def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
            super().__init__()
            out_features = out_features or in_features
            hidden_features = hidden_features or in_features
            self.fc1 = nn.Linear(in_features, hidden_features)
            self.act = act_layer()
            self.fc2 = nn.Linear(hidden_features, out_features)
            self.drop = nn.Dropout(drop)

        def forward(self, x):
            x = self.fc1(x)
            x = self.act(x)
            x = self.drop(x)
            x = self.fc2(x)
            x = self.drop(x)
            return x

    def window_partition(x, window_size):
        B, H, W, C = x.shape
        x = x.view(B, H // window_size, window_size, W // window_size, window_size, C)
        windows = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(-1, window_size, window_size, C)
        return windows

    def window_reverse(windows, window_size, H, W):
        B = int(windows.shape[0] / (H * W / window_size / window_size))
        x = windows.view(B, H // window_size, W // window_size, window_size, window_size, -1)
        x = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(B, H, W, -1)
        return x

    class WindowAttention(nn.Module):
        def __init__(self, dim, window_size, num_heads, qkv_bias=True, qk_scale=None, attn_drop=0., proj_drop=0.):
            super().__init__()
            self.dim = dim
            self.window_size = window_size
            self.num_heads = num_heads
            head_dim = dim // num_heads
            self.scale = qk_scale or head_dim ** -0.5

            self.relative_position_bias_table = nn.Parameter(
                torch.zeros((2 * window_size[0] - 1) * (2 * window_size[1] - 1), num_heads))

            coords_h = torch.arange(self.window_size[0])
            coords_w = torch.arange(self.window_size[1])
            coords = torch.stack(torch.meshgrid([coords_h, coords_w]))
            coords_flatten = torch.flatten(coords, 1)
            relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]
            relative_coords = relative_coords.permute(1, 2, 0).contiguous()
            relative_coords[:, :, 0] += self.window_size[0] - 1
            relative_coords[:, :, 1] += self.window_size[1] - 1
            relative_coords[:, :, 0] *= 2 * self.window_size[1] - 1
            relative_position_index = relative_coords.sum(-1)
            self.register_buffer("relative_position_index", relative_position_index)

            self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
            self.attn_drop = nn.Dropout(attn_drop)
            self.proj = nn.Linear(dim, dim)
            self.proj_drop = nn.Dropout(proj_drop)

            trunc_normal_(self.relative_position_bias_table, std=.02)
            self.softmax = nn.Softmax(dim=-1)

        def forward(self, x, mask=None):
            B_, N, C = x.shape
            qkv = self.qkv(x).reshape(B_, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
            q, k, v = qkv[0], qkv[1], qkv[2]

            q = q * self.scale
            attn = (q @ k.transpose(-2, -1))

            relative_position_bias = self.relative_position_bias_table[self.relative_position_index.view(-1)].view(
                self.window_size[0] * self.window_size[1], self.window_size[0] * self.window_size[1], -1)
            relative_position_bias = relative_position_bias.permute(2, 0, 1).contiguous()
            attn = attn + relative_position_bias.unsqueeze(0)

            if mask is not None:
                nW = mask.shape[0]
                attn = attn.view(B_ // nW, nW, self.num_heads, N, N) + mask.unsqueeze(1).unsqueeze(0)
                attn = attn.view(-1, self.num_heads, N, N)
                attn = self.softmax(attn)
            else:
                attn = self.softmax(attn)

            attn = self.attn_drop(attn)

            x = (attn @ v).transpose(1, 2).reshape(B_, N, C)
            x = self.proj(x)
            x = self.proj_drop(x)
            return x

        def extra_repr(self) -> str:
            return f'dim={self.dim}, window_size={self.window_size}, num_heads={self.num_heads}'
        
        def flops(self, N):
        # calculate flops for 1 window with token length of N
        flops = 0
        # qkv = self.qkv(x)
        flops += N * self.dim * 3 * self.dim
        # attn = (q @ k.transpose(-2, -1))
        flops += self.num_heads * N * (self.dim // self.num_heads) * N
        #  x = (attn @ v)
        flops += self.num_heads * N * N * (self.dim // self.num_heads)
        # x = self.proj(x)
        flops += N * self.dim * self.dim
        return flops


class SwinTransformerBlock(nn.Module):
    r""" Swin Transformer Block.

    Args:
        dim (int): Number of input channels.
        input_resolution (tuple[int]): Input resulotion.
        num_heads (int): Number of attention heads.
        window_size (int): Window size.
        shift_size (int): Shift size for SW-MSA.
        mlp_ratio (float): Ratio of mlp hidden dim to embedding dim.
        qkv_bias (bool, optional): If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set.
        drop (float, optional): Dropout rate. Default: 0.0
        attn_drop (float, optional): Attention dropout rate. Default: 0.0
        drop_path (float, optional): Stochastic depth rate. Default: 0.0
        act_layer (nn.Module, optional): Activation layer. Default: nn.GELU
        norm_layer (nn.Module, optional): Normalization layer.  Default: nn.LayerNorm
        fused_window_process (bool, optional): If True, use one kernel to fused window shift & window partition for acceleration, similar for the reversed part. Default: False
    """

    def __init__(self, dim, input_resolution, num_heads, window_size=7, shift_size=0,
                 mlp_ratio=4., qkv_bias=True, qk_scale=None, drop=0., attn_drop=0., drop_path=0.,
                 act_layer=nn.GELU, norm_layer=nn.LayerNorm,
                 fused_window_process=False):
        super().__init__()
        self.dim = dim
        self.input_resolution = input_resolution
        self.num_heads = num_heads
        self.window_size = window_size
        self.shift_size = shift_size
        self.mlp_ratio = mlp_ratio
        if min(self.input_resolution) <= self.window_size:
            # if window size is larger than input resolution, we don't partition windows
            self.shift_size = 0
            self.window_size = min(self.input_resolution)
        assert 0 <= self.shift_size < self.window_size, "shift_size must in 0-window_size"

        self.norm1 = norm_layer(dim)
        self.attn = WindowAttention(
            dim, window_size=to_2tuple(self.window_size), num_heads=num_heads,
            qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)

        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

        if self.shift_size > 0:
            # calculate attention mask for SW-MSA
            H, W = self.input_resolution
            img_mask = torch.zeros((1, H, W, 1))  # 1 H W 1
            h_slices = (slice(0, -self.window_size),
                        slice(-self.window_size, -self.shift_size),
                        slice(-self.shift_size, None))
            w_slices = (slice(0, -self.window_size),
                        slice(-self.window_size, -self.shift_size),
                        slice(-self.shift_size, None))
            cnt = 0
            for h in h_slices:
                for w in w_slices:
                    img_mask[:, h, w, :] = cnt
                    cnt += 1

            mask_windows = window_partition(img_mask, self.window_size)  # nW, window_size, window_size, 1
            mask_windows = mask_windows.view(-1, self.window_size * self.window_size)
            attn_mask = mask_windows.unsqueeze(1) - mask_windows.unsqueeze(2)
            attn_mask = attn_mask.masked_fill(attn_mask != 0, float(-100.0)).masked_fill(attn_mask == 0, float(0.0))
        else:
            attn_mask = None

        self.register_buffer("attn_mask", attn_mask)
        self.fused_window_process = fused_window_process

    def forward(self, x):
        H, W = self.input_resolution
        B, L, C = x.shape
        assert L == H * W, "input feature has wrong size"

        shortcut = x
        x = self.norm1(x)
        x = x.view(B, H, W, C)

        # cyclic shift
        if self.shift_size > 0:
            if not self.fused_window_process:
                shifted_x = torch.roll(x, shifts=(-self.shift_size, -self.shift_size), dims=(1, 2))
                # partition windows
                x_windows = window_partition(shifted_x, self.window_size)  # nW*B, window_size, window_size, C
            else:
                x_windows = WindowProcess.apply(x, B, H, W, C, -self.shift_size, self.window_size)
        else:
            shifted_x = x
            # partition windows
            x_windows = window_partition(shifted_x, self.window_size)  # nW*B, window_size, window_size, C

        x_windows = x_windows.view(-1, self.window_size * self.window_size, C)  # nW*B, window_size*window_size, C

        # W-MSA/SW-MSA
        attn_windows = self.attn(x_windows, mask=self.attn_mask)  # nW*B, window_size*window_size, C

        # merge windows
        attn_windows = attn_windows.view(-1, self.window_size, self.window_size, C)

        # reverse cyclic shift
        if self.shift_size > 0:
            if not self.fused_window_process:
                shifted_x = window_reverse(attn_windows, self.window_size, H, W)  # B H' W' C
                x = torch.roll(shifted_x, shifts=(self.shift_size, self.shift_size), dims=(1, 2))
            else:
                x = WindowProcessReverse.apply(attn_windows, B, H, W, C, self.shift_size, self.window_size)
        else:
            shifted_x = window_reverse(attn_windows, self.window_size, H, W)  # B H' W' C
            x = shifted_x
        x = x.view(B, H * W, C)
        x = shortcut + self.drop_path(x)

        # FFN
        x = x + self.drop_path(self.mlp(self.norm2(x)))

        return x

    def extra_repr(self) -> str:
        return f"dim={self.dim}, input_resolution={self.input_resolution}, num_heads={self.num_heads}, " \
               f"window_size={self.window_size}, shift_size={self.shift_size}, mlp_ratio={self.mlp_ratio}"

    def flops(self):
        flops = 0
        H, W = self.input_resolution
        # norm1
        flops += self.dim * H * W
        # W-MSA/SW-MSA
        nW = H * W / self.window_size / self.window_size
        flops += nW * self.attn.flops(self.window_size * self.window_size)
        # mlp
        flops += 2 * H * W * self.dim * self.dim * self.mlp_ratio
        # norm2
        flops += self.dim * H * W
        return flops


class PatchMerging(nn.Module):
    r""" Patch Merging Layer.

    Args:
        input_resolution (tuple[int]): Resolution of input feature.
        dim (int): Number of input channels.
        norm_layer (nn.Module, optional): Normalization layer.  Default: nn.LayerNorm
    """

    def __init__(self, input_resolution, dim, norm_layer=nn.LayerNorm):
        super().__init__()
        self.input_resolution = input_resolution
        self.dim = dim
        self.reduction = nn.Linear(4 * dim, 2 * dim, bias=False)
        self.norm = norm_layer(4 * dim)

    def forward(self, x):
        """
        x: B, H*W, C
        """
        H, W = self.input_resolution
        B, L, C = x.shape
        assert L == H * W, "input feature has wrong size"
        assert H % 2 == 0 and W % 2 == 0, f"x size ({H}*{W}) are not even."

        x = x.view(B, H, W, C)

        x0 = x[:, 0::2, 0::2, :]  # B H/2 W/2 C
        x1 = x[:, 1::2, 0::2, :]  # B H/2 W/2 C
        x2 = x[:, 0::2, 1::2, :]  # B H/2 W/2 C
        x3 = x[:, 1::2, 1::2, :]  # B H/2 W/2 C
        x = torch.cat([x0, x1, x2, x3], -1)  # B H/2 W/2 4*C
        x = x.view(B, -1, 4 * C)  # B H/2*W/2 4*C

        x = self.norm(x)
        x = self.reduction(x)

        return x

    def extra_repr(self) -> str:
        return f"input_resolution={self.input_resolution}, dim={self.dim}"

    def flops(self):
        H, W = self.input_resolution
        flops = H * W * self.dim
        flops += (H // 2) * (W // 2) * 4 * self.dim * 2 * self.dim
        return flops


class BasicLayer(nn.Module):
    """ A basic Swin Transformer layer for one stage.

    Args:
        dim (int): Number of input channels.
        input_resolution (tuple[int]): Input resolution.
        depth (int): Number of blocks.
        num_heads (int): Number of attention heads.
        window_size (int): Local window size.
        mlp_ratio (float): Ratio of mlp hidden dim to embedding dim.
        qkv_bias (bool, optional): If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set.
        drop (float, optional): Dropout rate. Default: 0.0
        attn_drop (float, optional): Attention dropout rate. Default: 0.0
        drop_path (float | tuple[float], optional): Stochastic depth rate. Default: 0.0
        norm_layer (nn.Module, optional): Normalization layer. Default: nn.LayerNorm
        downsample (nn.Module | None, optional): Downsample layer at the end of the layer. Default: None
        use_checkpoint (bool): Whether to use checkpointing to save memory. Default: False.
        fused_window_process (bool, optional): If True, use one kernel to fused window shift & window partition for acceleration, similar for the reversed part. Default: False
    """

    def __init__(self, dim, input_resolution, depth, num_heads, window_size,
                 mlp_ratio=4., qkv_bias=True, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., norm_layer=nn.LayerNorm, downsample=None, use_checkpoint=False,
                 fused_window_process=False):

        super().__init__()
        self.dim = dim
        self.input_resolution = input_resolution
        self.depth = depth
        self.use_checkpoint = use_checkpoint

        # build blocks
        self.blocks = nn.ModuleList([
            SwinTransformerBlock(dim=dim, input_resolution=input_resolution,
                                 num_heads=num_heads, window_size=window_size,
                                 shift_size=0 if (i % 2 == 0) else window_size // 2,
                                 mlp_ratio=mlp_ratio,
                                 qkv_bias=qkv_bias, qk_scale=qk_scale,
                                 drop=drop, attn_drop=attn_drop,
                                 drop_path=drop_path[i] if isinstance(drop_path, list) else drop_path,
                                 norm_layer=norm_layer,
                                 fused_window_process=fused_window_process)
            for i in range(depth)])

        # patch merging layer
        if downsample is not None:
            self.downsample = downsample(input_resolution, dim=dim, norm_layer=norm_layer)
        else:
            self.downsample = None

    def forward(self, x):
        for blk in self.blocks:
            if self.use_checkpoint:
                x = checkpoint.checkpoint(blk, x)
            else:
                x = blk(x)
        if self.downsample is not None:
            x = self.downsample(x)
        return x

    def extra_repr(self) -> str:
        return f"dim={self.dim}, input_resolution={self.input_resolution}, depth={self.depth}"

    def flops(self):
        flops = 0
        for blk in self.blocks:
            flops += blk.flops()
        if self.downsample is not None:
            flops += self.downsample.flops()
        return flops


class PatchEmbed(nn.Module):
    r""" Image to Patch Embedding

    Args:
        img_size (int): Image size.  Default: 224.
        patch_size (int): Patch token size. Default: 4.
        in_chans (int): Number of input image channels. Default: 3.
        embed_dim (int): Number of linear projection output channels. Default: 96.
        norm_layer (nn.Module, optional): Normalization layer. Default: None
    """

    def __init__(self, img_size=224, patch_size=4, in_chans=3, embed_dim=96, norm_layer=None):
        super().__init__()
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        patches_resolution = [img_size[0] // patch_size[0], img_size[1] // patch_size[1]]
        self.img_size = img_size
        self.patch_size = patch_size
        self.patches_resolution = patches_resolution
        self.num_patches = patches_resolution[0] * patches_resolution[1]

        self.in_chans = in_chans
        self.embed_dim = embed_dim

        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)
        if norm_layer is not None:
            self.norm = norm_layer(embed_dim)
        else:
            self.norm = None

    def forward(self, x):
        B, C, H, W = x.shape
        # FIXME look at relaxing size constraints
        assert H == self.img_size[0] and W == self.img_size[1], \
            f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]})."
        x = self.proj(x).flatten(2).transpose(1, 2)  # B Ph*Pw C
        if self.norm is not None:
            x = self.norm(x)
        return x

    def flops(self):
        Ho, Wo = self.patches_resolution
        flops = Ho * Wo * self.embed_dim * self.in_chans * (self.patch_size[0] * self.patch_size[1])
        if self.norm is not None:
            flops += Ho * Wo * self.embed_dim
        return flops


class SwinTransformer(nn.Module):
    r""" Swin Transformer
        A PyTorch impl of : `Swin Transformer: Hierarchical Vision Transformer using Shifted Windows`  -
          https://arxiv.org/pdf/2103.14030

    Args:
        img_size (int | tuple(int)): Input image size. Default 224
        patch_size (int | tuple(int)): Patch size. Default: 4
        in_chans (int): Number of input image channels. Default: 3
        num_classes (int): Number of classes for classification head. Default: 1000
        embed_dim (int): Patch embedding dimension. Default: 96
        depths (tuple(int)): Depth of each Swin Transformer layer.
        num_heads (tuple(int)): Number of attention heads in different layers.
        window_size (int): Window size. Default: 7
        mlp_ratio (float): Ratio of mlp hidden dim to embedding dim. Default: 4
        qkv_bias (bool): If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float): Override default qk scale of head_dim ** -0.5 if set. Default: None
        drop_rate (float): Dropout rate. Default: 0
        attn_drop_rate (float): Attention dropout rate. Default: 0
        drop_path_rate (float): Stochastic depth rate. Default: 0.1
        norm_layer (nn.Module): Normalization layer. Default: nn.LayerNorm.
        ape (bool): If True, add absolute position embedding to the patch embedding. Default: False
        patch_norm (bool): If True, add normalization after patch embedding. Default: True
        use_checkpoint (bool): Whether to use checkpointing to save memory. Default: False
        fused_window_process (bool, optional): If True, use one kernel to fused window shift & window partition for acceleration, similar for the reversed part. Default: False
    """

    def __init__(self, img_size=224, patch_size=4, in_chans=3, num_classes=1000,
                 embed_dim=96, depths=[2, 2, 6, 2], num_heads=[3, 6, 12, 24],
                 window_size=7, mlp_ratio=4., qkv_bias=True, qk_scale=None,
                 drop_rate=0., attn_drop_rate=0., drop_path_rate=0.1,
                 norm_layer=nn.LayerNorm, ape=False, patch_norm=True,
                 use_checkpoint=False, fused_window_process=False, **kwargs):
        super().__init__()

        self.num_classes = num_classes
        self.num_layers = len(depths)
        self.embed_dim = embed_dim
        self.ape = ape
        self.patch_norm = patch_norm
        self.num_features = int(embed_dim * 2 ** (self.num_layers - 1))
        self.mlp_ratio = mlp_ratio

        # split image into non-overlapping patches
        self.patch_embed = PatchEmbed(
            img_size=img_size, patch_size=patch_size, in_chans=in_chans, embed_dim=embed_dim,
            norm_layer=norm_layer if self.patch_norm else None)
        num_patches = self.patch_embed.num_patches
        patches_resolution = self.patch_embed.patches_resolution
        self.patches_resolution = patches_resolution

        # absolute position embedding
        if self.ape:
            self.absolute_pos_embed = nn.Parameter(torch.zeros(1, num_patches, embed_dim))
            trunc_normal_(self.absolute_pos_embed, std=.02)

        self.pos_drop = nn.Dropout(p=drop_rate)

        # stochastic depth
        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))]  # stochastic depth decay rule

        # build layers
        self.layers = nn.ModuleList()
        for i_layer in range(self.num_layers):
            layer = BasicLayer(dim=int(embed_dim * 2 ** i_layer),
                               input_resolution=(patches_resolution[0] // (2 ** i_layer),
                                                 patches_resolution[1] // (2 ** i_layer)),
                               depth=depths[i_layer],
                               num_heads=num_heads[i_layer],
                               window_size=window_size,
                               mlp_ratio=self.mlp_ratio,
                               qkv_bias=qkv_bias, qk_scale=qk_scale,
                               drop=drop_rate, attn_drop=attn_drop_rate,
                               drop_path=dpr[sum(depths[:i_layer]):sum(depths[:i_layer + 1])],
                               norm_layer=norm_layer,
                               downsample=PatchMerging if (i_layer < self.num_layers - 1) else None,
                               use_checkpoint=use_checkpoint,
                               fused_window_process=fused_window_process)
            self.layers.append(layer)

        self.norm = norm_layer(self.num_features)
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.head = nn.Linear(self.num_features, num_classes) if num_classes > 0 else nn.Identity()

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    @torch.jit.ignore
    def no_weight_decay(self):
        return {'absolute_pos_embed'}

    @torch.jit.ignore
    def no_weight_decay_keywords(self):
        return {'relative_position_bias_table'}

    def forward_features(self, x):
        x = self.patch_embed(x)
        if self.ape:
            x = x + self.absolute_pos_embed
        x = self.pos_drop(x)

        for layer in self.layers:
            x = layer(x)

        x = self.norm(x)  # B L C
        x = self.avgpool(x.transpose(1, 2))  # B C 1
        x = torch.flatten(x, 1)
        return x

    def forward(self, x):
        x = self.forward_features(x)
        x = self.head(x)
        return x

    def flops(self):
        flops = 0
        flops += self.patch_embed.flops()
        for i, layer in enumerate(self.layers):
            flops += layer.flops()
        flops += self.num_features * self.patches_resolution[0] * self.patches_resolution[1] // (2 ** self.num_layers)
        flops += self.num_features * self.num_classes
        return flops
    
    # For now, let's just apply some Conv layers to simulate processing
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(inputs)
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)

    return x  # Modify this return statement based on your intended output


IndentationError: expected an indented block after function definition on line 119 (804458131.py, line 121)

# New Concept

In [7]:
import os
import glob
import numpy as np
import tensorflow as tf
from skimage.io import imread
from skimage import img_as_float
from tensorflow.keras.layers import (
    Input, Conv2D, Add, Lambda, LayerNormalization, Dense, MultiHeadAttention, DepthwiseConv2D
)
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load and preprocess images from a given folder
def load_images_from_folder(folder, size=(128, 128)):
    images = []
    for filename in glob.glob(os.path.join(folder, '*.png')):
        img = imread(filename)
        img = img_as_float(img)
        img_resized = tf.image.resize(img, size).numpy()
        images.append(img_resized)
    return np.array(images)

# Data preprocessing and augmentation pipeline
def preprocess_and_augment_data(images, augment=False):
    if augment:
        datagen = ImageDataGenerator(
            rotation_range=15,
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0.01,
            zoom_range=[0.9, 1.25],
            horizontal_flip=True,
            fill_mode='reflect'
        )
        augmented_images = datagen.flow(images, batch_size=len(images), shuffle=False)
        augmented_images = next(augmented_images)  # Fetch the augmented images
        return augmented_images
    return images

# Function to load datasets from base folder (for both training and testing)
def load_and_preprocess_datasets(base_folder, size=(128, 128), augment=False):
    rainy_images_train, clear_images_train = [], []
    rainy_images_test, clear_images_test = [], []

    datasets = ['Rain200L', 'Rain200H']  # Add more datasets if necessary

    for dataset in datasets:
        train_folder = os.path.join(base_folder, dataset, 'train')
        test_folder = os.path.join(base_folder, dataset, 'test')

        # Load training data
        rainy_train_folder = os.path.join(train_folder, 'input')
        clear_train_folder = os.path.join(train_folder, 'target')
        rainy_images_train.extend(load_images_from_folder(rainy_train_folder, size))
        clear_images_train.extend(load_images_from_folder(clear_train_folder, size))

        # Load testing data
        rainy_test_folder = os.path.join(test_folder, 'input')
        clear_test_folder = os.path.join(test_folder, 'target')
        rainy_images_test.extend(load_images_from_folder(rainy_test_folder, size))
        clear_images_test.extend(load_images_from_folder(clear_test_folder, size))

    # Convert lists to numpy arrays
    rainy_images_train = np.array(rainy_images_train)
    clear_images_train = np.array(clear_images_train)
    rainy_images_test = np.array(rainy_images_test)
    clear_images_test = np.array(clear_images_test)

    # Apply data augmentation to training data (if augment is True)
    if augment:
        rainy_images_train = preprocess_and_augment_data(rainy_images_train, augment=True)
        clear_images_train = preprocess_and_augment_data(clear_images_train, augment=True)

    return (rainy_images_train, clear_images_train), (rainy_images_test, clear_images_test)

# Advanced Dynamic Pyramid Model
def advanced_dynamic_pyramid_model(input_shape, num_scales=3):
    inputs = Input(shape=input_shape)
    pyramid_features = []

    for scale in range(num_scales):
        scale_factor = 2 ** scale
        
        # Resizing each input for different scales
        x_scaled = Lambda(lambda x: tf.image.resize(x, 
                            [input_shape[0] // scale_factor, input_shape[1] // scale_factor]))(inputs)
        
        # Dynamic kernel size based on scale
        kernel_size = 3 + scale
        
        # Use Depthwise Separable Convolutions
        x_conv = DepthwiseConv2D((kernel_size, kernel_size), padding='same', activation='relu')(x_scaled)
        x_conv = Conv2D(64 * scale_factor, (1, 1), padding='same')(x_conv)  # Pointwise Conv
        
        # Residual connection
        x_scaled_conv = Conv2D(64 * scale_factor, (1, 1), padding='same')(x_scaled)
        x_conv = Add()([x_conv, x_scaled_conv])
        
        # Resize back to original input size before concatenation
        x_resized = Lambda(lambda x: tf.image.resize(x, 
                              [input_shape[0], input_shape[1]]))(x_conv)
        
        pyramid_features.append(x_resized)

    # Concatenate all pyramid features along channel axis
    fused_features = Lambda(
        lambda x: tf.concat(x, axis=-1),
    )(pyramid_features)
    
    return inputs, fused_features

# Swin Transformer Block Implementation with k-th Attention
class SwinTransformerBlock(tf.keras.layers.Layer):
    def __init__(self, dim, num_heads, window_size, k=4, shift_size=0, mlp_ratio=4.):
        super(SwinTransformerBlock, self).__init__()
        self.window_size = window_size
        self.shift_size = shift_size
        self.num_heads = num_heads
        self.k = k  # Number of top-k attention weights
        self.dim = dim
        
        # Layer normalization
        self.norm1 = LayerNormalization(epsilon=1e-5)
        self.norm2 = LayerNormalization(epsilon=1e-5)
        
        # Multi-head self-attention
        self.attn = MultiHeadAttention(num_heads=num_heads, key_dim=dim)
        
        # Feed-forward network
        self.mlp = self.get_mlp(dim, mlp_ratio)

    def get_mlp(self, dim, mlp_ratio):
        return tf.keras.Sequential([
            Dense(int(dim * mlp_ratio), activation='relu'),
            Dense(dim)
        ])
    
    def build(self, input_shape):
        pass  # You can initialize any additional weights here if necessary

    def call(self, x):
        # Layer normalization
        x_norm = self.norm1(x)

        # Compute attention
        attn_output = self.attn(x_norm, x_norm)

        # Get top-k attention weights
        attn_weights = tf.nn.softmax(tf.matmul(x_norm, x_norm, transpose_b=True))  # Computing attention weights
        top_k_values, indices = tf.nn.top_k(attn_weights, k=self.k)
        threshold = tf.reduce_min(top_k_values, axis=-1, keepdims=True)

        # Suppress weights below the threshold
        attn_weights = tf.where(attn_weights < threshold, tf.zeros_like(attn_weights), attn_weights)

        # Normalize the weights again after thresholding
        attn_weights = attn_weights / tf.reduce_sum(attn_weights, axis=-1, keepdims=True)

        # Use the modified attention weights to compute output
        attn_output = tf.matmul(attn_weights, attn_output)

        # Skip connection
        x = x + attn_output  

        # Feed-forward network
        x = self.norm2(x)
        mlp_output = self.mlp(x)
        x = x + mlp_output  # Skip connection
        
        return x

def swin_transformer_block(inputs):
    # Define parameters for the Swin Transformer Block
    dim = inputs.shape[-1]  # Input channel dimension
    num_heads = 4  # Number of attention heads
    window_size = 7  # Size of the window
    shift_size = 0  # Shift size for the windowing scheme
    
    # Create the Swin Transformer Block
    x = SwinTransformerBlock(dim, num_heads, window_size, k=4, shift_size=shift_size)(inputs)
    
    return x

# Recursive Swin Transformer Block with residual connections
def recursive_swin_transformer_block(inputs, num_recursions=6):
    x = inputs
    for _ in range(num_recursions):
        # Call the Swin Transformer block
        x = swin_transformer_block(x)
    
    return x

# Image Restoration and Enhancement Block
def image_restoration_and_enhancement(inputs, num_classes=3):
    # Normalize the input features
    x = LayerNormalization()(inputs)
    
    # Using Depthwise Separable Convolutions
    x = DepthwiseConv2D((3, 3), padding='same', activation='relu')(x)
    x = Conv2D(32, (1, 1), padding='same', activation='relu')(x)  # Pointwise Conv

    x = DepthwiseConv2D((3, 3), padding='same', activation='relu')(x)
    x = Conv2D(32, (1, 1), padding='same', activation='relu')(x)  # Pointwise Conv

    x = DepthwiseConv2D((3, 3), padding='same', activation='relu')(x)
    x = Conv2D(num_classes, (1, 1), padding='same', activation='sigmoid')(x)  # Final output layer

    return x

# Build the full model
def build_full_model(input_shape=(128, 128, 3)):
    inputs, pyramid_features = advanced_dynamic_pyramid_model(input_shape=input_shape, num_scales=3)

    # Create the recursive model for feature extraction
    transformer_output = recursive_swin_transformer_block(pyramid_features, num_recursions=6)

    # Image restoration and enhancement
    outputs = image_restoration_and_enhancement(transformer_output, num_classes=input_shape[2])  # Keeping the output channels same as input channels

    # Create and return the model
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Set parameters
base_folder = '/kaggle/input/derainingdata/RainData'  # Update with your dataset path
input_shape = (128, 128, 3)

# Load and preprocess the dataset
(rainy_train, clear_train), (rainy_test, clear_test) = load_and_preprocess_datasets(base_folder, size=(128, 128), augment=True)

# Build and compile the model
model = build_full_model(input_shape=input_shape)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Print the model summary
model.summary()

# Optionally, you can train the model
# model.fit(rainy_train, clear_train, epochs=50, batch_size=32, validation_split=0.1)


Model: "functional_21"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_16      │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_28 (Lambda)  │ (None, 128, 128,  │          0 │ input_layer_16[0… │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_30 (Lambda)  │ (None, 64, 64, 3) │          0 │ input_layer_16[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_32 (Lambda)  │ (None, 32, 32, 3) │          0 │ input_layer_16[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv2d_6  │ (None, 128, 128,  │         30 │ lambda_28[0][0]   │
│ (DepthwiseConv2D)   │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv2d_7  │ (None, 64, 64, 3) │         51 │ lambda_30[0][0]   │
│ (DepthwiseConv2D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv2d_8  │ (None, 32, 32, 3) │         78 │ lambda_32[0][0]   │
│ (DepthwiseConv2D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_29 (Conv2D)  │ (None, 128, 128,  │        256 │ depthwise_conv2d… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_30 (Conv2D)  │ (None, 128, 128,  │        256 │ lambda_28[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_31 (Conv2D)  │ (None, 64, 64,    │        512 │ depthwise_conv2d… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_32 (Conv2D)  │ (None, 64, 64,    │        512 │ lambda_30[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_33 (Conv2D)  │ (None, 32, 32,    │      1,024 │ depthwise_conv2d… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_34 (Conv2D)  │ (None, 32, 32,    │      1,024 │ lambda_32[0][0]   │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_18 (Add)        │ (None, 128, 128,  │          0 │ conv2d_29[0][0],  │
│                     │ 64)               │            │ conv2d_30[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_19 (Add)        │ (None, 64, 64,    │          0 │ conv2d_31[0][0],  │
│                     │ 128)              │            │ conv2d_32[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_20 (Add)        │ (None, 32, 32,    │          0 │ conv2d_33[0][0],  │
│                     │ 256)              │            │ conv2d_34[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_29 (Lambda)  │ (None, 128, 128,  │          0 │ add_18[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 28,985,794 (110.57 MB)

 Trainable params: 28,985,794 (110.57 MB)

 Non-trainable params: 0 (0.00 B)

# Training

In [8]:
# Training the model
history = model.fit(rainy_train, clear_train, epochs=50, batch_size=32, validation_split=0.1)

# Function to calculate PSNR
def calculate_psnr(target, ref):
    return 20 * np.log10(1.0 / np.sqrt(mean_squared_error(target.flatten(), ref.flatten())))

# Evaluate the model
def evaluate_model(model, rainy_test, clear_test):
    predictions = model.predict(rainy_test)

    psnr_values = []
    ssim_values = []
    
    for i in range(len(predictions)):
        psnr = calculate_psnr(clear_test[i], predictions[i])
        ssim_value = ssim(clear_test[i], predictions[i], multichannel=True)
        psnr_values.append(psnr)
        ssim_values.append(ssim_value)

    avg_psnr = np.mean(psnr_values)
    avg_ssim = np.mean(ssim_values)

    return predictions, avg_psnr, avg_ssim

# Evaluating the model
predictions, avg_psnr, avg_ssim = evaluate_model(model, rainy_test, clear_test)
print(f"Average PSNR: {avg_psnr:.2f} dB")
print(f"Average SSIM: {avg_ssim:.4f}")

# Plotting results
def plot_results(rainy_images, clear_images, predicted_images, n=5):
    plt.figure(figsize=(15, 5))
    for i in range(n):
        plt.subplot(3, n, i + 1)
        plt.imshow(rainy_images[i])
        plt.title("Rainy Image")
        plt.axis("off")

        plt.subplot(3, n, i + 1 + n)
        plt.imshow(clear_images[i])
        plt.title("Clear Image")
        plt.axis("off")

        plt.subplot(3, n, i + 1 + 2 * n)
        plt.imshow(predicted_images[i])
        plt.title("Predicted Image")
        plt.axis("off")
    
    plt.tight_layout()
    plt.show()

# Plotting some results
plot_results(rainy_test, clear_test, predictions)

Epoch 1/50


I0000 00:00:1727293061.240601      98 service.cc:145] XLA service 0x7b91e4093f70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727293061.240658      98 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2024-09-25 19:37:53.202426: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng18{k11=0} for conv (f32[448,1,3,3]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,448,128,128]{3,2,1,0}, f32[32,448,128,128]{3,2,1,0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, feature_group_count=448, custom_call_target="__cudnn$convBackwardFilter", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-09-25 19:37:53.618573: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.416237209s

ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code

  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/opt/conda/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/opt/conda/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelbase.py", line 545, in di

#  Import Libraries

In [20]:
# Torch imports
import torch
import torch.nn as nn
import torch.cuda as cuda
import torch.optim as optim
from torchvision import datasets, transforms
from timm.models.layers import trunc_normal_
from torch.utils.data import DataLoader
from torchsummary import summary

# Other imports
import numpy as np
import glob
import os
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from skimage.io import imread
from skimage import img_as_float
import warnings
import matplotlib.pyplot as plt

# Own files import
from transformer_block import TransformerBlock, PatchEmbed, PatchUnEmbed
from Data.data_loader import Rain800TrainData, Rain800ValData
from my_utils import batch_PSNR, batch_SSIM, output_to_image
from my_utils import save_ckp, load_ckp, base_path

# Hyperparameters
training_image_size = 56
dtype = torch.cuda.FloatTensor
batch_size = 5
torch.manual_seed(1234)
torch.cuda.manual_seed_all(1234)
epochs = 4600
lr = 0.0001
error_plot_freq = 20
INT_MAX = 2147483647
error_tolerence = 10
patch_size = 1

# Paths
base_pth = base_path()
ckp_pth = base_pth + "/CheckPoints"

# Miscellaneous
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (16, 9)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Load and preprocess images from a given folder
def load_images_from_folder(folder, size=(128, 128)):
    images = []
    for filename in glob.glob(os.path.join(folder, '*.png')):
        img = imread(filename)
        img = img_as_float(img)
        img_resized = tf.image.resize(img, size).numpy()
        images.append(img_resized)
    return np.array(images)

# Data preprocessing and augmentation pipeline
def preprocess_and_augment_data(images, augment=False):
    if augment:
        datagen = ImageDataGenerator(
            rotation_range=15,
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0.01,
            zoom_range=[0.9, 1.25],
            horizontal_flip=True,
            fill_mode='reflect'
        )
        augmented_images = datagen.flow(images, batch_size=len(images), shuffle=False)
        augmented_images = next(augmented_images)  # Fetch the augmented images
        return augmented_images
    return images

# Function to load datasets from base folder (for both training and testing)
def load_and_preprocess_datasets(base_folder, size=(128, 128), augment=False):
    rainy_images_train, clear_images_train = [], []
    rainy_images_test, clear_images_test = [], []

    datasets = ['Rain200L', 'Rain200H']

    for dataset in datasets:
        train_folder = os.path.join(base_folder, dataset, 'train')
        test_folder = os.path.join(base_folder, dataset, 'test')

        # Load training data
        rainy_train_folder = os.path.join(train_folder, 'input')
        clear_train_folder = os.path.join(train_folder, 'target')
        rainy_images_train.extend(load_images_from_folder(rainy_train_folder, size))
        clear_images_train.extend(load_images_from_folder(clear_train_folder, size))

        # Load testing data
        rainy_test_folder = os.path.join(test_folder, 'input')
        clear_test_folder = os.path.join(test_folder, 'target')
        rainy_images_test.extend(load_images_from_folder(rainy_test_folder, size))
        clear_images_test.extend(load_images_from_folder(clear_test_folder, size))

    rainy_images_train = np.array(rainy_images_train)
    clear_images_train = np.array(clear_images_train)
    rainy_images_test = np.array(rainy_images_test)
    clear_images_test = np.array(clear_images_test)

    if augment:
        rainy_images_train = preprocess_and_augment_data(rainy_images_train, augment=True)
        clear_images_train = preprocess_and_augment_data(clear_images_train, augment=True)

    return (rainy_images_train, clear_images_train), (rainy_images_test, clear_images_test)

# Basic Block for Transformer
class BasicBlock(nn.Module):
    def __init__(self, dim, input_resolution, num_heads, patch_size):
        super().__init__()
        self.dim = dim
        self.patch_size = patch_size
        self.input_resolution = input_resolution
        H, W = self.input_resolution
        self.num_heads = num_heads
        self.transformer = TransformerBlock(self.dim, (H // self.patch_size, W // self.patch_size), num_heads)

    def forward(self, x):
        x = self.transformer(x)
        return x

# Residual Layer
class ResidualLayer(nn.Module):
    def __init__(self, dim, input_resolution, residual_depth, patch_size):
        super().__init__()
        self.dim = dim
        self.patch_size = patch_size
        self.residual_depth = residual_depth
        self.input_resolution = input_resolution
        self.block1 = BasicBlock(self.dim, self.input_resolution, 2, self.patch_size)
        self.block2 = BasicBlock(self.dim, self.input_resolution, 2, self.patch_size)
        self.conv_out = nn.Conv2d(self.dim, self.dim, 3, padding=1)

    def forward(self, x):
        B, HW, C = x.shape
        H, W = self.input_resolution
        shortcut = x
        for _ in range(self.residual_depth):
            x = self.block1(self.block2(x))
            x = torch.add(x, shortcut)
        x = x.transpose(1, 2).view(B, C, H // self.patch_size, W // self.patch_size)
        x = self.conv_out(x).flatten(2).transpose(1, 2)  # B L C
        return x

# Deep Recursive Transformer Model
class DeepRecursiveTransformer(nn.Module):
    def __init__(self, dim, input_resolution, patch_size, residual_depth, recursive_depth):
        super().__init__()
        self.dim = dim
        self.patch_size = patch_size
        self.recursive_depth = recursive_depth
        self.input_resolution = input_resolution
        self.H, self.W = self.input_resolution
        assert self.H == self.W, "Input height and width should be the same"
        self.input_conv1 = nn.Conv2d(3, self.dim, 3, padding=1)
        self.patch_embed = PatchEmbed(img_size=self.H, patch_size=self.patch_size, in_chans=3, embed_dim=self.dim)
        self.patch_unembed = PatchUnEmbed(img_size=self.H, patch_size=self.patch_size, in_chans=self.dim, unembed_dim=3)
        self.recursive_layers = nn.ModuleList()
        for i in range(self.recursive_depth):
            layer = ResidualLayer(self.dim, self.input_resolution, self.residual_depth, self.patch_size)
            self.recursive_layers.append(layer)
        self.output_conv1 = nn.Conv2d(self.dim, 3, 3, padding=1)
        self.normalise_layer = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        self.denormalise_layer = transforms.Normalize((-0.485, -0.456, -0.406), (1. / 0.229, 1. / 0.224, 1. / 0.225))
        self.apply(self._init_weights)
        self.activation = nn.LeakyReLU()

    def _init_weights(self, l):
        if isinstance(l, nn.Linear):
            trunc_normal_(l.weight, std=.02)
            if isinstance(l, nn.Linear) and l.bias is not None:
                nn.init.constant_(l.bias, 0)
        elif isinstance(l, nn.LayerNorm):
            nn.init.constant_(l.bias, 0)
            nn.init.constant_(l.weight, 1.0)

    def forward(self, x):
        x = self.normalise_layer(x)
        outer_shortcut = x
        x = self.patch_embed(x)
        inner_shortcut = x

        for i in range(len(self.recursive_layers)):
            x = self.recursive_layers[i](x)

        x = torch.add(x, inner_shortcut)
        x = self.patch_unembed(x, (self.H // self.patch_size, self.W // self.patch_size))
        x = torch.add(x, outer_shortcut)
        x = self.denormalise_layer(x)
        return x  # output shape (B, C, H, W)

# Load datasets
base_folder = '/kaggle/input/derainingdata/RainData'  # Update this path
(train_rainy, train_clear), (test_rainy, test_clear) = load_and_preprocess_datasets(base_folder, size=(training_image_size, training_image_size), augment=True)

# Initialize the model
net = DeepRecursiveTransformer(96, (training_image_size, training_image_size), patch_size, 3, 6)
summary(net.cuda(), (3, training_image_size, training_image_size))


ModuleNotFoundError: No module named 'torchsummary'

# Image Preprocessing

In [2]:
import os
import glob
import numpy as np
import tensorflow as tf
from skimage.io import imread
from skimage import img_as_float
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load and preprocess images from a given folder
def load_images_from_folder(folder, size=(128, 128)):
    images = []
    for filename in glob.glob(os.path.join(folder, '*.png')):
        img = imread(filename)
        img = img_as_float(img)
        img_resized = tf.image.resize(img, size).numpy()
        images.append(img_resized)
    return np.array(images)

# Data preprocessing and augmentation pipeline
def preprocess_and_augment_data(images, augment=False):
    if augment:
        datagen = ImageDataGenerator(
            rotation_range=15,
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0.01,
            zoom_range=[0.9, 1.25],
            horizontal_flip=True,
            fill_mode='reflect'
        )
        augmented_images = datagen.flow(images, batch_size=len(images), shuffle=False)
        augmented_images = next(augmented_images)  # Fetch the augmented images
        return augmented_images
    return images

# Function to load datasets from base folder (for both training and testing)
def load_and_preprocess_datasets(base_folder, size=(128, 128), augment=False):
    rainy_images_train, clear_images_train = [], []
    rainy_images_test, clear_images_test = [], []  # Corrected initialization

    datasets = ['Rain200L', 'Rain200H']  # Add more datasets if necessary

    for dataset in datasets:
        train_folder = os.path.join(base_folder, dataset, 'train')
        test_folder = os.path.join(base_folder, dataset, 'test')

        # Load training data
        rainy_train_folder = os.path.join(train_folder, 'input')
        clear_train_folder = os.path.join(train_folder, 'target')
        rainy_images_train.extend(load_images_from_folder(rainy_train_folder, size))
        clear_images_train.extend(load_images_from_folder(clear_train_folder, size))

        # Load testing data
        rainy_test_folder = os.path.join(test_folder, 'input')
        clear_test_folder = os.path.join(test_folder, 'target')
        rainy_images_test.extend(load_images_from_folder(rainy_test_folder, size))
        clear_images_test.extend(load_images_from_folder(clear_test_folder, size))

    # Convert lists to numpy arrays
    rainy_images_train = np.array(rainy_images_train)
    clear_images_train = np.array(clear_images_train)
    rainy_images_test = np.array(rainy_images_test)
    clear_images_test = np.array(clear_images_test)

    # Apply data augmentation to training data (if augment is True)
    if augment:
        rainy_images_train = preprocess_and_augment_data(rainy_images_train, augment=True)
        clear_images_train = preprocess_and_augment_data(clear_images_train, augment=True)

    return (rainy_images_train, clear_images_train), (rainy_images_test, clear_images_test)

# Example usage
base_folder = '/kaggle/input/derainingdata/RainData'
(train_rainy, train_clear), (test_rainy, test_clear) = load_and_preprocess_datasets(base_folder, size=(128, 128), augment=True)

print(f"Training Rainy Images Shape: {train_rainy.shape}")
print(f"Training Clear Images Shape: {train_clear.shape}")
print(f"Testing Rainy Images Shape: {test_rainy.shape}")
print(f"Testing Clear Images Shape: {test_clear.shape}")


Training Rainy Images Shape: (3600, 128, 128, 3)
Training Clear Images Shape: (3600, 128, 128, 3)
Testing Rainy Images Shape: (400, 128, 128, 3)
Testing Clear Images Shape: (400, 128, 128, 3)


# Advanced Dynamic Pyramid Model

In [6]:
import tensorflow as tf
import tensorflow_probability as tfp  # Import TensorFlow Probability for percentile calculation
from keras.layers import Input, Conv2D, Add, Lambda, LayerNormalization, MultiHeadAttention, Dense, Layer, UpSampling2D
from tensorflow.keras.models import Model

# Advanced Dynamic Pyramid Model
def advanced_dynamic_pyramid_model(input_shape, num_scales=3):
    inputs = Input(shape=input_shape)
    pyramid_features = []

    for scale in range(num_scales):
        scale_factor = 2 ** scale
        
        # Resizing each input for different scales
        x_scaled = Lambda(lambda x: tf.image.resize(x, 
                            [input_shape[0] // scale_factor, input_shape[1] // scale_factor]))(inputs)
        
        # Dynamic kernel size based on scale
        kernel_size = 3 + scale
        x_conv = Conv2D(64 * scale_factor, (kernel_size, kernel_size), activation='relu', padding='same')(x_scaled)
        
        # Residual connection
        x_scaled_conv = Conv2D(64 * scale_factor, (1, 1), padding='same')(x_scaled)
        x_conv = Add()([x_conv, x_scaled_conv])
        
        # Resize back to original input size before concatenation
        x_resized = Lambda(lambda x: tf.image.resize(x, 
                              [input_shape[0], input_shape[1]]))(x_conv)
        
        pyramid_features.append(x_resized)

    # Concatenate all pyramid features along channel axis
    fused_features = Lambda(
        lambda x: tf.concat(x, axis=-1),
    )(pyramid_features)
    
    return inputs, fused_features

# Custom Layer: Window Partition Layer
class WindowPartitionLayer(Layer):
    def __init__(self, window_size):
        super(WindowPartitionLayer, self).__init__()
        self.window_size = window_size

    def call(self, inputs):
        batch_size, height, width, channels = tf.shape(inputs)[0], tf.shape(inputs)[1], tf.shape(inputs)[2], tf.shape(inputs)[3]
        windowed = tf.image.extract_patches(
            inputs,
            sizes=[1, self.window_size, self.window_size, 1],
            strides=[1, self.window_size, self.window_size, 1],
            rates=[1, 1, 1, 1],
            padding='VALID'
        )
        windowed = tf.reshape(windowed, (batch_size, -1, self.window_size * self.window_size, channels))
        return windowed

    def compute_output_shape(self, input_shape):
        return (input_shape[0], (input_shape[1] // self.window_size) * (input_shape[2] // self.window_size), self.window_size * self.window_size, input_shape[3])

# Custom Layer: Patch Merge Layer
class PatchMergeLayer(Layer):
    def __init__(self, window_size):
        super(PatchMergeLayer, self).__init__()
        self.window_size = window_size

    def call(self, inputs):
        batch_size, num_windows, flattened_window_size, channels = tf.shape(inputs)[0], tf.shape(inputs)[1], tf.shape(inputs)[2], tf.shape(inputs)[3]
        merged = tf.reshape(inputs, (batch_size, int(num_windows**0.5), int(num_windows**0.5), self.window_size, self.window_size, channels))
        merged = tf.transpose(merged, perm=[0, 1, 3, 2, 4, 5])
        merged = tf.reshape(merged, (batch_size, merged.shape[1] * self.window_size, merged.shape[3] * self.window_size, channels))
        return merged, self.window_size * 2

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1] * 2, input_shape[2] * 2, input_shape[3])

# Modified Swin Transformer Block with dynamic attention filtering
class ModifiedSwinTransformerBlock(Layer):
    def __init__(self, initial_window_size=4, num_heads=4, key_dim=64, num_recursions=3):
        super(ModifiedSwinTransformerBlock, self).__init__()
        self.initial_window_size = initial_window_size
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.num_recursions = num_recursions

    def call(self, fused_features):
        original_shape = tf.shape(fused_features)
        return self.recursive_block(fused_features, self.initial_window_size, 2, self.num_recursions, original_shape)

    def recursive_block(self, x, window_size, iteration, recursion, original_shape):
        if recursion == 0:
            return x
        
        for _ in range(iteration):
            # Partition the window
            partition_layer = WindowPartitionLayer(window_size)
            windows = partition_layer(x)

            window_shape = (tf.shape(windows)[-2], tf.shape(windows)[-1])
            windows_reshaped = tf.reshape(windows, (-1, window_shape[0] * window_shape[1], tf.shape(x)[-1]))

            # Apply LayerNormalization and MultiHeadAttention
            x_norm = LayerNormalization(axis=-1)(windows_reshaped)
            attention = MultiHeadAttention(num_heads=self.num_heads, key_dim=self.key_dim)(x_norm, x_norm)

            # Dynamically filter 60-80% attention values based on a threshold
            attention_scores = tf.reduce_mean(tf.abs(attention), axis=-1, keepdims=True)
            threshold = tfp.stats.percentile(attention_scores, tf.random.uniform([], 60, 80))
            attention = tf.where(attention_scores > threshold, attention, tf.zeros_like(attention))

            # Residual connection
            x_add = Add()([windows_reshaped, attention])
            x_reconstructed = self.window_reverse(x_add, original_shape)

            # Apply LayerNormalization and FFN
            x_norm_ffn = LayerNormalization(axis=-1)(x_reconstructed)
            x_ffn = Dense(128, activation='relu')(x_norm_ffn)
            x_ffn_out = Dense(tf.shape(x)[-1])(x_ffn)

            x_out = Add()([x_reconstructed, x_ffn_out])

            # Merge the patches and enlarge window size for next iteration
            patch_merge_layer = PatchMergeLayer(window_size)
            x_out, window_size = patch_merge_layer(x_out)

        return self.recursive_block(x_out, window_size, iteration, recursion - 1, original_shape)

    def window_reverse(self, x, original_shape):
        batch_size = tf.shape(x)[0]
        num_windows = tf.shape(x)[1] // (self.initial_window_size * self.initial_window_size)
        x = tf.reshape(x, (batch_size, num_windows, self.initial_window_size, self.initial_window_size, -1))
        return tf.reshape(x, (batch_size, original_shape[1], original_shape[2], -1))

    def compute_output_shape(self, input_shape):
        # The output shape changes based on recursion and merging, so return the computed shape
        return input_shape  # You may need to adjust this if shape transforms across layers

# Image Restoration and Enhancement Block
def image_restoration_and_enhancement(x_out, num_classes=3):
    x = LayerNormalization()(x_out)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = LayerNormalization()(x)
    outputs = Conv2D(num_classes, (3, 3), padding='same', activation='tanh')(x)
    return outputs

# Full Model Construction
def build_full_model(input_shape):
    inputs, pyramid_features = advanced_dynamic_pyramid_model(input_shape=input_shape, num_scales=3)
    transformer_block = ModifiedSwinTransformerBlock(initial_window_size=4, num_heads=4, key_dim=64, num_recursions=3)
    transformer_output = transformer_block(pyramid_features)
    outputs = image_restoration_and_enhancement(x_out=transformer_output, num_classes=3)
    model = Model(inputs=inputs, outputs=outputs)
    
    return model

# Compile the model
input_shape = (256, 256, 3)  # Example input shape
model = build_full_model(input_shape)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Summary of the model
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_14 (Lambda)  │ (None, 256, 256,  │          0 │ input_layer_2[0]… │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_16 (Lambda)  │ (None, 128, 128,  │          0 │ input_layer_2[0]… │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_18 (Lambda)  │ (None, 64, 64, 3) │          0 │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_12 (Conv2D)  │ (None, 256, 256,  │      1,792 │ lambda_14[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_13 (Conv2D)  │ (None, 256, 256,  │        256 │ lambda_14[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_14 (Conv2D)  │ (None, 128, 128,  │      6,272 │ lambda_16[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_15 (Conv2D)  │ (None, 128, 128,  │        512 │ lambda_16[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_16 (Conv2D)  │ (None, 64, 64,    │     19,456 │ lambda_18[0][0]   │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_17 (Conv2D)  │ (None, 64, 64,    │      1,024 │ lambda_18[0][0]   │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_7 (Add)         │ (None, 256, 256,  │          0 │ conv2d_12[0][0],  │
│                     │ 64)               │            │ conv2d_13[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_8 (Add)         │ (None, 128, 128,  │          0 │ conv2d_14[0][0],  │
│                     │ 128)              │            │ conv2d_15[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_9 (Add)         │ (None, 64, 64,    │          0 │ conv2d_16[0][0],  │
│                     │ 256)              │            │ conv2d_17[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_15 (Lambda)  │ (None, 256, 256,  │          0 │ add_7[0][0]       │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_17 (Lambda)  │ (None, 256, 256,  │          0 │ add_8[0][0]       │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_19 (Lambda)  │ (None, 256, 256,  │          0 │ add_9[0][0]       │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_20 (Lambda)  │ (None, 256, 256,  │          0 │ lambda_15[0][0],  │
│                     │ 448)              │            │ lambda_17[0][0],

 Total params: 402,819 (1.54 MB)

 Trainable params: 402,819 (1.54 MB)

 Non-trainable params: 0 (0.00 B)

# Window Partition and Reverse Functions

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

class WindowPartitionLayer(Layer):
    def __init__(self, window_size):
        super(WindowPartitionLayer, self).__init__()
        self.window_size = window_size

    def call(self, inputs):
        patches = tf.image.extract_patches(images=inputs,
                                           sizes=[1, self.window_size, self.window_size, 1],
                                           strides=[1, self.window_size, self.window_size, 1],
                                           rates=[1, 1, 1, 1],
                                           padding='VALID')
        return patches

def window_partition(x, window_size):
    partition_layer = WindowPartitionLayer(window_size)
    return partition_layer(x)


# Patch Merging Function

In [ ]:
# Patch Merging Function
def patch_merge(x, window_size):
    """Merge adjacent patches to create a larger feature window."""
    partition_layer = WindowPartitionLayer(window_size)
    patches = partition_layer(x)

    # Calculate the new merged patch (by averaging or pooling)
    merged_patches = tf.reduce_mean(patches, axis=-1, keepdims=True)

    # Reshape into the original window size
    new_window_size = window_size * 2
    merged = window_reverse(merged_patches, new_window_size, x.shape)

    return merged, new_window_size

# Modified Swin Transformer Block with Recursion and Iteration

In [ ]:
from tensorflow.keras.layers import Layer

def modified_swin_transformer_block(fused_features, initial_window_size, num_heads, key_dim, num_recursions):
    # Define a recursive block as a Keras layer
    class RecursiveBlock(Layer):
        def __init__(self, window_size, num_heads, key_dim, num_recursions):
            super().__init__()
            self.window_size = window_size
            self.num_heads = num_heads
            self.key_dim = key_dim
            self.num_recursions = num_recursions

        def call(self, x):
            if self.num_recursions <= 0:
                return x
            
            # Partition the input into windows
            windows = window_partition(x, self.window_size)
            window_shape = (windows.shape[-2], windows.shape[-1])
            windows_reshaped = tf.reshape(windows, (-1, window_shape[0] * window_shape[1], x.shape[-1]))

            # Layer Normalization before Attention
            x_norm = LayerNormalization()(windows_reshaped)
            attention = MultiHeadAttention(num_heads=self.num_heads, key_dim=self.key_dim)(x_norm, x_norm)

            # Apply reverse window operation
            x_out = window_reverse(attention, window_shape, x.shape)
            x_out, window_size = patch_merge(x_out, window_size)

            # Recur on the output
            return RecursiveBlock(window_size, self.num_heads, self.key_dim, self.num_recursions - 1)(x_out)

    # Instantiate and call the recursive block
    recursive_layer = RecursiveBlock(initial_window_size, num_heads, key_dim, num_recursions)
    return recursive_layer(fused_features)


# Image Restoration and Enhancement

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import LayerNormalization, Conv2D, UpSampling2D

def image_restoration_and_enhancement(x_out, num_classes=3):
    x = LayerNormalization()(x_out)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = LayerNormalization()(x)
    outputs = Conv2D(num_classes, (3, 3), padding='same', activation='tanh')(x)
    return outputs


# Construct and Compile the Model

In [ ]:
from tensorflow.keras.models import Model

def build_full_model(input_shape):
    inputs = Input(shape=input_shape)
    inputs, pyramid_features = advanced_dynamic_pyramid_model(input_shape=input_shape, num_scales=3)
    transformer_output = modified_swin_transformer_block(fused_features=pyramid_features, 
                                                         initial_window_size=4, num_heads=4, key_dim=64, num_recursions=6)
    outputs = image_restoration_and_enhancement(x_out=transformer_output, num_classes=3)
    model = Model(inputs=inputs, outputs=outputs)
    return model

input_shape = (256, 256, 3)
model = build_full_model(input_shape)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model.summary()


# Model Training

In [1]:
# Import necessary libraries for training
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt

# Load and preprocess datasets, resizing images to (256, 256)
(train_rainy, train_clear), (test_rainy, test_clear) = load_and_preprocess_datasets(base_folder, size=(256, 256), augment=True)

# Assign training and validation data
train_data = train_rainy  # Input for training
train_labels = train_clear  # Target/Label for training

val_data = test_rainy  # Input for validation
val_labels = test_clear  # Target/Label for validation

# Set up input shape and define the model
input_shape = (256, 256, 3)
model = build_full_model(input_shape)

# Compile the model with Adam optimizer and MSE loss
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Define callbacks for saving the best model and early stopping
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Define training parameters
epochs = 50
batch_size = 16

# Train the model
history = model.fit(train_data, train_labels,
                    validation_data=(val_data, val_labels),
                    epochs=epochs,
                    batch_size=batch_size,
                    callbacks=[checkpoint, early_stopping])

# Save the final model
model.save('final_model.keras')

# Optionally, plot the training and validation loss over epochs
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


NameError: name 'load_and_preprocess_datasets' is not defined

# Model Evaluation

In [ ]:
# Import necessary libraries for evaluation
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model('best_model.h5')

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_data, test_labels)

# Print the test results
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


# PSNR and SSIM Calculation

In [ ]:
# Import necessary libraries for PSNR and SSIM calculations
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import numpy as np

# Function to calculate PSNR
def calculate_psnr(ground_truth, prediction):
    return peak_signal_noise_ratio(ground_truth, prediction, data_range=1.0)

# Function to calculate SSIM
def calculate_ssim(ground_truth, prediction):
    return structural_similarity(ground_truth, prediction, multichannel=True, data_range=1.0)

# Use the trained model to predict on the test data
predictions = model.predict(test_data)

# Initialize lists to store PSNR and SSIM values
psnr_list = []
ssim_list = []

# Loop through each test image and calculate PSNR and SSIM
for i in range(len(test_data)):
    gt_image = test_labels[i]
    pred_image = predictions[i]
    
    psnr = calculate_psnr(gt_image, pred_image)
    ssim = calculate_ssim(gt_image, pred_image)
    
    psnr_list.append(psnr)
    ssim_list.append(ssim)

# Calculate average PSNR and SSIM
average_psnr = np.mean(psnr_list)
average_ssim = np.mean(ssim_list)

# Print the results
print(f"Average PSNR: {average_psnr:.4f}")
print(f"Average SSIM: {average_ssim:.4f}")


# Prediction and Evaluation on Test Images

In [ ]:
# Import necessary libraries for visualization
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('best_model.h5')

# Predict on test data
predictions = model.predict(test_data)

# Visualize the results
num_images_to_display = 3

for i in range(num_images_to_display):
    gt_image = test_labels[i]
    pred_image = predictions[i]
    
    # Display ground truth and prediction side by side
    plt.figure(figsize=(10, 5))
    
    plt.subplot(1, 2, 1)
    plt.title("Ground Truth")
    plt.imshow((gt_image * 255).astype(np.uint8))  # Convert to range [0, 255] for display
    
    plt.subplot(1, 2, 2)
    plt.title("Predicted")
    plt.imshow((pred_image * 255).astype(np.uint8))  # Convert to range [0, 255] for display
    
    plt.show()


# PSNR and SSIM Evaluation Across Test Set

In [ ]:
# PSNR and SSIM Evaluation Across the Test Set
import numpy as np
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

# Function to evaluate PSNR and SSIM for the entire test set
def evaluate_test_set_psnr_ssim(test_data, test_labels, predictions):
    psnr_values = []
    ssim_values = []

    for i in range(len(test_data)):
        gt_image = test_labels[i]
        pred_image = predictions[i]
        
        psnr = peak_signal_noise_ratio(gt_image, pred_image, data_range=1.0)
        ssim = structural_similarity(gt_image, pred_image, multichannel=True, data_range=1.0)
        
        psnr_values.append(psnr)
        ssim_values.append(ssim)

    # Compute the average PSNR and SSIM
    avg_psnr = np.mean(psnr_values)
    avg_ssim = np.mean(ssim_values)

    print(f"Average PSNR: {avg_psnr:.4f}")
    print(f"Average SSIM: {avg_ssim:.4f}")
    
    return avg_psnr, avg_ssim

# Get predictions from the model
predictions = model.predict(test_data)

# Evaluate PSNR and SSIM on the test set
avg_psnr, avg_ssim = evaluate_test_set_psnr_ssim(test_data, test_labels, predictions)


# Visualization of Training History

In [ ]:
# Import libraries for visualization
import matplotlib.pyplot as plt

# Plot the training and validation loss
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot the training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Show the plots
plt.show()
